In [1]:
import os

import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pandas as pd

import commons, geometry, utils

In [3]:
Lstart = 50
Lx = 470
Ly = 470
Lz = 60
Rp = 6
eps_se = 0.5
eps_am = 1 - eps_se
markers = commons.SurfaceMarkers()
phases = commons.Phases()
CELL_TYPES = commons.CellTypes()
start_zpos = 50
scale = (800/470) * 1e-6
scale_factor = [scale, scale, scale]
outdir = "mesh/laminate/"
utils.make_dir_if_missing(outdir)
msh_path = os.path.join(outdir, 'laminate.msh')
tetr_path = os.path.join(outdir, 'tetr.xdmf')
tria_path = os.path.join(outdir, 'tria.xdmf')

## Build Laminate Geometry of Cylindrical SE Strands

In [4]:
df = 470 * pd.read_csv('data/laminate.csv')

In [ ]:
gmsh.initialize()
gmsh.model.add('laminate')
gmsh.option.setNumber('General.Verbosity', 1)
box_se = gmsh.model.occ.addBox(0, 0, Lstart, Lx, Ly, Lz)
gmsh.model.occ.synchronize()
box_am = gmsh.model.occ.addBox(0, 0, Lstart + Lz, Lx, Ly, Rp)
gmsh.model.occ.synchronize()
cylinders = []

for idx in range(df.shape[0]):
    x, y, _ = df.loc[idx, :]
    if (x + Rp >= 470) or (y + Rp >= 470):
        continue
    if (x - Rp <= 0) or (y - Rp <= 0):
        continue
    cyl = gmsh.model.occ.addCylinder(x, y, Lstart + Rp, x, y, Lstart + Lz + Rp, Rp)
    cylinders.append(cyl)
    gmsh.model.occ.synchronize()

se_phase = gmsh.model.occ.cut([(3, box_se)], [(3, c) for c in cylinders], removeTool=False)
gmsh.model.occ.synchronize()
union = gmsh.model.occ.fuse([(3, box_am)], [(3, c) for c in cylinders])
gmsh.model.occ.synchronize()

vols = gmsh.model.occ.getEntities(3)
se_vol = gmsh.model.addPhysicalGroup(3, [vols[0][1]], phases.electrolyte)
am_vol = gmsh.model.addPhysicalGroup(3, [vols[1][1]], phases.active_material)
gmsh.model.occ.synchronize()

right = []
left = []
insulated = []
interface = []
for surf in gmsh.model.occ.getEntities(2):
    com = gmsh.model.occ.getCenterOfMass(surf[0], surf[1])
    if np.isclose(com[2], Lstart + Lz + Rp):
        right.append(surf[1])
    elif np.isclose(com[2], Lstart + Lz + 0.5 * Rp):
        if np.isclose(com[1], 235) or np.isclose(com[0], 235):
            insulated.append(surf[1])
        else:
            interface.append(surf[1])
    elif np.isclose(com[2], Lstart):
        left.append(surf[1])
    else:
        interface.append(surf[1])

gmsh.model.addPhysicalGroup(2, left, markers.left_cc)
gmsh.model.addPhysicalGroup(2, right, markers.right_cc)
gmsh.model.addPhysicalGroup(2, insulated, markers.insulated)
gmsh.model.addPhysicalGroup(2, interface, markers.am_se_interface)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(msh_path)
gmsh.finalize()

In [32]:
msh = meshio.read(msh_path)
tetr_unscaled = geometry.create_mesh(msh, CELL_TYPES.tetra)
tetr_unscaled.write(tetr_path)
tetr_scaled = geometry.scale_mesh(tetr_unscaled, CELL_TYPES.tetra, scale_factor=scale_factor)
tetr_scaled.write(tetr_path)
tria_unscaled = geometry.create_mesh(msh, CELL_TYPES.triangle)
tria_unscaled.write(tria_path)
tria_scaled = geometry.scale_mesh(tria_unscaled, CELL_TYPES.triangle, scale_factor=scale_factor)
tria_scaled.write(tria_path)


$Element section not found.


Error: Couldn't read file mesh/laminate/laminate.msh as either of ansys, gmsh

SystemExit: 1